Pretende-se a criação de um protótipo em Sagemath para o algoritmo KYBER: implementação de um KEM, que seja IND-CPA seguro, e um PKE que seja IND-CCA seguro.

Tal como apresentado [aqui](https://www.dropbox.com/sh/mx4bybl0d6e9g1m/AAAJsNOarzJ0fApTi8aBwzXVa/Kyber-20201001/Supporting_Documentation?dl=0&preview=kyber.pdf&subfolder_nav_tracking=1), o KYBER é IND-CCA2 seguro. 
Uma vez que o CCA2-KEM foi concebido para proporcionar um nível de segurança mais elevado do que o CCA-KEM, também pode ser considerado IND-CPA seguro. Isto deve-se ao facto de que qualquer ataque que quebre a segurança CPA, quebra também a segurança CCA2. No entanto, uma vez que é apenas pretendido um protótipo cujo KEM seja IND-CPA seguro, com base num esquema PKE IND-CCA seguro, a transformação de Fujisaki-Okamoto (FO), tal como está nos apontamentos da disciplina, será aplicada no esquema PKE. Note-se que a implementação do KEM-CCA2 conforme o artigo já mencionado segue uma variante da transformação FO, mas neste trabalho a trasnformação FO é aplicada ao PKE.

**Notação** 

$R$ corresponde ao anel $\mathbb{Z}[X]/(X^n+1)$ e $R_q$ ao anel $\mathbb{Z}_q[X]/(X^n+1)$ 

In [1]:
from cryptography.hazmat.primitives import hashes
from pickle import dumps, loads

In [2]:
# Classe que implementa as multiplicações em R - number-theoretic transform (NTT) 
class NTT:

    def __init__(self, n=128, q=None):
        
        if not  n in [32,64,128,256,512,1024,2048]:
            raise ValueError("improper argument ",n)
        self.n = n  
        if not q:
            self.q = 1 + 2*n
            while True:
                if (self.q).is_prime():
                    break
                self.q += 2*n
        else:
            if q % (2*n) != 1:
                raise ValueError("Valor de 'q' não verifica a condição NTT")
            self.q = q
             
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="w")
        w = (self.R).gen()
        
        g = (w^n + 1)
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs]) 
    
    def ntt(self,f):
        
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ; ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * xi2                     
            return ff
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
    def invNtt(self,ff):                             
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
# Operações sobre matrizes e vetores
# Soma de matrizes
def sumMatrix(e1, e2, n):
    for i in range(len(e1)):
        e1[i] = sumVector(e1[i], e2[i], n)
    return e1

# Subtração de matrizes
def subMatrix(e1, e2, n):
    for i in range(len(e1)):
        e1[i] = subVector(e1[i], e2[i], n)
    return e1

# Multiplicação de matrizes
def multMatrix(vec1, vec2, n):
    for i in range(len(vec1)):
        vec1[i] = multVector(vec1[i], vec2[i],n)
    tmp = [0] * n
    for i in range(len(vec1)):
        tmp = sumVector(tmp, vec1[i], n)
    return tmp

# Multiplicação de uma matriz por um vector
def multMatrixVector(M, v, k, n) :
    for i in range(len(M)):
        for j in range(len(M[i])):
            M[i][j] = multVector(M[i][j], v[j], n)
    tmp = [[0] * n] * k 
    for i in range(len(M)):
        for j in range(len(M[i])):
            tmp[i] = sumVector(tmp[i], M[i][j],n)
    return tmp

# Soma de vetores
def sumVector(ff1, ff2, n):
    res = []
    for i in range(n):
        res.append((ff1[i] + ff2[i]))
    return res

# Multiplicação de vetores
def multVector(ff1, ff2, n):
    res = []
    for i in range(n):
        res.append((ff1[i] * ff2[i]))
    return res

# Subtração de vetores
def subVector(ff1, ff2, n):
    res = []
    for i in range(n):
        res.append((ff1[i] - ff2[i]))
    return res

# **Implementação do esquema PKE IND-CCA seguro**

Conforme já mencionado, o esquema PKE apresentado será sujeito à transformação FO de modo a ser IND-CCA seguro. A geração das chaves não sofre qualquer alteração, pelo que está de acordo com o algoritmo 4. Os métodos para cifrar e decifrar serão alterados conforme a transformação FO.

**KeyGen (algoritmo 4)**

São geradas a chave privada e a chave pública. O método começa por gerar a matriz $Â \in R_q^{k \times k}$ no domínio NTT e os vetores $s, e ∈ R_q$. Para tal foram utilizadas os métodos *Parse* e *CBD* implementados de acordo com os algoritmos 1 e 2, respetivamente. Conforme apresentado no algoritmo 4, são geradas as variáveis necessárias para calcular a chave pública $pk = (Â * s + e, \rho)$, com  $\rho = G(d)[32:]$ e $d$ um valor pseudo-aleatório, e a chave privada $sk = s$.

**Encrypt (algoritmo 5)**

Cifra, com recurso à chave pública e um valor aleatório, uma mensagem *m* gerada a partir do anel $R_q$. O processo está descrito em pseudo-código no algoritmo 5 do artigo.

De modo a ser um PKE IND-CCA seguro, a transformação FO indica que o método de cifra passará a ser:
$$E'(x)\;\equiv\;\vartheta\,r\gets h\,\centerdot\,\vartheta\,(y,r') \gets (x\oplus g(r)\,,\, h(r\|y))\,\centerdot\,(y\,,\,f(r,r'))$$

Começa-se por gerar o valor aleatório $r \in R_q$. Calcula-se $y$ como a operação de XOR entre o *plaintext* e o hash do valor $r$, com recurso à primitiva SHA3_256. Além disso, $r$ é misturado com $y$ para construir via o hash $h$ (que neste caso será também a primitiva SHA3_256) uma nova fonte de aleatoriedade $r'=h(r\|y)$.
Finalmente, o resultado é o par formado pela ofuscação $y$ e o criptograma que resulta de, com o $f$ original, i.e. o método *encrypt* original, cifrar $r$ com a aleatoriedade $r'$.

**Decrypt (algoritmo 6)**

Decifra, com recurso à chave privada, o *ciphertext*. Novamente, o processo está descrito no artigo, agora no algoritmo 6.

Por fim, e de forma a garantir segurança IND-CCA do PKE, o método será transformado via a transformação FO de acordo com a expressão:
$$D'(y,c)\;\equiv\;\vartheta\,r \gets D(c)\,\centerdot\,\mathsf{if}\;\;c\neq f(r,h(r\|y))\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$
Começa-se por decifrar o criptograma $c$ com recurso ao método original *decrypt*. O resultado é usado para derivar a ofuscação da chave tal como no método de cifra, utilizando o método original *encrypt*, de forma a verificar se a ofuscação recebida é igual. Se for igual, a chave é válida e procede-se ao XOR do $y$ com o hash do criptograma incial já decifrado.

In [3]:
class KYBER_PKE:
    
    def __init__(self, pset):
        self.n, self.q, self.T, self.k, self.n1, self.n2, self.du, self.dv, self.Rq = self.setup(pset)
    
    def setup(self, pset):
        n = 256
        q = 7681
        n2 = 2
        if pset == 512:
            k = 2
            n1 = 3
            du = 10
            dv = 4
        elif pset == 768:
            k = 3
            n1 = 2
            du = 10
            dv = 4
        elif pset == 1024:
            k = 4
            n1 = 2
            du = 11
            dv = 5
        else: print("Error: Parameter set not valid!")
            
        Zq.<w> = GF(q)[]
        fi = w^n + 1
        Rq.<w> = QuotientRing(Zq, Zq.ideal(fi))
        
        T = NTT(n,q)
        
        return n, q, T, k, n1, n2, du, dv, Rq
    
    def bytes2Bits(self, byteArray):
        bitArray = []
        for elem in byteArray:
            bitElemArr = []
            for i in range(0,8): 
                bitElemArr.append(mod(elem//2**(mod(i,8)),2))
                for i in range(0,len(bitElemArr)):
                    bitArray.append(bitElemArr[i])
        return bitArray
    
    def G(self, h):
        digest = hashes.Hash(hashes.SHA3_512())
        digest.update(bytes(h))
        g = digest.finalize()
        return g[:32],g[32:]
    
    def XOF(self,b,b1,b2):
        digest = hashes.Hash(hashes.SHAKE128(int(self.q)))
        digest.update(b)
        digest.update(bytes(b1))
        digest.update(bytes(b2))
        m = digest.finalize()
        return m
    
    def PRF(self,b,b1): 
        digest = hashes.Hash(hashes.SHAKE256(int(self.q)))
        digest.update(b)
        digest.update(bytes(b1))
        return digest.finalize()

    def Compress(self,x,d) :
        coefficients = x.list()
        newCoefficients = []
        for c in coefficients:
            new = mod(round( int(2 ** d) / self.q * int(c)), int(2 ** d))
            newCoefficients.append(new)
        return self.Rq(newCoefficients)
    
    def Decompress(self,x,d) :
        coefficients = x.list()
        newCoefficients = []
        for c in coefficients:
            new = round(self.q / (2 ** d) * int(c))
            newCoefficients.append(new)
        return self.Rq(newCoefficients)
    
    # Método XOR
    def xor(self, b1, b2):
        return bytes(a ^^ b for a, b in zip(b1, b2))
    
    # Algorithm 1
    def Parse(self, byteArray):
        i = 0
        j = 0
        a = []
        while j < self.n:
            d1 = byteArray[i] + 256 * mod(byteArray[i+1],16)
            d2 = byteArray[i+1]//16 + 16 * byteArray[i+2]
            if d1 < self.q :
                a.append(d1)
                j = j+1
            if d2 < self.q and j<self.n:
                a.append(d2)
                j = j+1
            i = i+3
        return self.Rq(a)
    
    
    # Algorithm 2
    def CBD(self, byteArray, nn):
        f=[0]*self.n
        bitArray = self.bytes2Bits(byteArray)
        for i in range(256):
            a = 0
            b = 0
            for j in range(nn):
                a += bitArray[2*i*nn + j]
                b += bitArray[2*i*nn + nn + j]
            f[i] = a-b
        return self.Rq(f)
    
    # Algorithm 3
    def Decode(self, byteArray, l):
        f = []
        bitArray = self.bytes2Bits(byteArray)
        for i in range(len(byteArray)):
            fi = 0
            for j in range(l):
                fi += int(bitArray[i*l+j]) * 2**j
            f.append(fi)
        return self.Rq(f)
    
    # Algorithm 4
    def keyGen(self):
        d = bytearray(os.urandom(32))
        ro, sigma = self.G(d)
        N = 0
        
        # Generate matrix Â in Rq in NTT domain
        A = []
        for i in range(self.k):
            A.append([])
            for j in range(self.k):
                A[i].append(self.T.ntt(self.Parse(self.XOF(ro,j,i))))
        
        # Sample s in Rq from Bη1
        s = []  
        for i in range(self.k):
            s.insert(i,self.CBD(self.PRF(sigma,N), self.n1))
            N = N+1
            
        # Sample e in Rq from Bη1
        e = []
        for i in range(self.k):
            e.insert(i,self.CBD(self.PRF(sigma,N), self.n1))
            N = N+1

        for i in range(self.k) :
            s[i] = self.T.ntt(s[i])
            e[i] = self.T.ntt(e[i])
            
        t = sumMatrix(multMatrixVector(A,s,self.k,self.n), e, self.n)
        
        pk = t, ro
        sk = s
        
        return pk, sk
    
    # Algorithm 5
    def encrypt(self, pk, m, r):
        N = 0
        t, ro = pk
        
        # Generate matrix Â in Rq in NTT domain
        transposeA = []
        for i in range(self.k):
            transposeA.append([])
            for j in range(self.k):
                transposeA[i].append(self.T.ntt(self.Parse(self.XOF(ro,i,j))))
        
        # Sample r in Rq from Bη1
        rr = []
        for i in range(self.k):
            rr.insert(i,self.T.ntt(self.CBD(self.PRF(r, N), self.n1)))
            N += 1
        
        # Sample e1 in Rq from Bη2
        e1 = []
        for i in range(self.k):
            e1.insert(i,self.CBD(self.PRF(r, N), self.n2))
            N += 1
        
        # Sample e2 in Rq from Bη2
        e2 = self.CBD(self.PRF(r, N), self.n2)
        
        uAux = multMatrixVector(transposeA, rr, self.k, self.n)
        uAux2 = []
        for i in range(len(uAux)) :
            uAux2.append(self.T.invNtt(uAux[i]))
        uAux3 = sumMatrix(uAux2, e1, self.n)
        u = []
        for i in range(len(uAux3)) :
            u.append(self.Rq(uAux3[i]))
            
        vAux = multMatrix(t, rr, self.n)
        vAux1 = self.T.invNtt(vAux)
        vAux2 = self.Rq(sumVector(vAux1, e2, self.n))
        
        v = self.Rq(sumVector(vAux2, self.Decompress(m, 1), self.n))
        
        # Compress(u, du)
        c1 = []
        for i in range(len(u)):
            c1.append(self.Compress(u[i], self.du))
        
        # Compress(v, dv)
        c2 = self.Compress(v, self.dv)
        
        return c1, c2
    
    # Algorithm 6
    def decrypt(self, sk, c):
        c1, c2 = c
 
        u = []       
        for i in range(len(c1)):
            u.append(self.Decompress(c1[i], self.du))
        
        v = self.Decompress(c2,self.dv)

        s = sk
        
        uNTT = []
        for i in range(len(u)) :
            uNTT.append(self.T.ntt(u[i]))
        
        mAux = subVector(v, self.T.invNtt(multMatrix(s, uNTT, self.n)), self.n)

        m = self.Compress(self.Rq(mAux), 1)
        
        return m
    
    # hashes h e g
    def hashFOT(self, b):
        r = hashes.Hash(hashes.SHA3_256())
        r.update(b)
        return r.finalize()
    
    def encryptCCA(self, x, pk):
        r = self.Rq([choice([0, 1]) for i in range(self.n)])
        y = self.xor(x, self.hashFOT(bytes(r)))
        c = self.encrypt(pk, r, self.hashFOT(bytes(r)+y))
        return (y, c)

    def decryptCCA(self, y, c, pk, sk):
        r = self.decrypt(sk, c)
        derived_c = self.encrypt(pk, r, self.hashFOT(bytes(r)+y))
        if c[0] != derived_c[0]:
            print("Error: key doesn't match!")
            return None
        else:
            return self.xor(y, self.hashFOT(bytes(r)))

## **Teste da transformação FO**

In [4]:
kyber = KYBER_PKE(512)

pk, sk = kyber.keyGen()
m = b'Hello there!'
print("Original message:")
print(m)

y, c = kyber.encryptCCA(m, pk)
print("\nCiphertext:")
print(c)

plaintext = kyber.decryptCCA(y, c, pk, sk)
print("\nDecrypted ciphertext:")
print(plaintext)

Original message:
b'Hello there!'

Ciphertext:
([169*w^255 + 475*w^254 + 884*w^253 + 887*w^252 + 622*w^251 + 662*w^250 + 728*w^249 + 724*w^248 + 453*w^247 + 730*w^246 + 134*w^245 + 311*w^244 + 605*w^243 + 500*w^242 + 598*w^241 + 179*w^240 + 790*w^239 + 501*w^238 + 136*w^237 + 456*w^236 + 257*w^235 + 279*w^234 + 972*w^233 + 927*w^232 + 156*w^231 + 228*w^230 + 673*w^229 + 180*w^228 + 868*w^227 + 283*w^226 + 505*w^225 + 407*w^224 + 749*w^223 + 157*w^222 + 41*w^221 + 596*w^220 + 588*w^219 + 580*w^218 + 856*w^217 + 336*w^216 + 544*w^215 + 26*w^214 + 396*w^213 + 537*w^212 + 45*w^211 + 755*w^210 + 565*w^209 + 397*w^208 + 602*w^207 + 830*w^206 + 70*w^205 + 639*w^204 + 161*w^203 + 468*w^202 + 52*w^201 + 686*w^200 + 78*w^199 + 861*w^198 + 630*w^197 + 261*w^196 + 95*w^195 + 590*w^194 + 54*w^193 + 815*w^192 + 723*w^191 + 793*w^190 + 170*w^189 + 101*w^188 + 190*w^187 + 219*w^186 + 24*w^185 + 196*w^184 + 231*w^183 + 396*w^182 + 970*w^181 + 379*w^180 + 206*w^179 + 876*w^178 + 548*w^177 + 100*w^176 + 

# **Implementação do KEM IND-CPA-secure**

**KeyGen**

Foi utilizada a função *keyGen* definida na classe KYBER_PKE.

**Encapsulamento da chave**

Começa-se por calcular o *hash* de um nonce aleatório $m \in R_q$ que irá ser a chave secreta. Por fim, segue-se a cifragem do $m$ (convertido em bytes) pela função *encryptCCA* com recurso à chave pública, obtendo o criptograma e uma ofuscação da chave.

**Desencapsulamento da chave**

Utilizando a chave privada, utiliza-se a função *decryptCCA* com o criptograma e a ofuscação da chave de modo a obter o valor aleatório $m$. Por fim, calcula-se o hash do $m$.

In [5]:
class KYBER_KEM:
    def __init__(self, pset):
        self.pke = self.setup(pset)
        
    def setup(self, pset):
        pke = KYBER_PKE(pset)
        return pke
    
    def H(self, b):
        r = hashes.Hash(hashes.SHA3_256())
        r.update(b)
        return r.finalize()
    
    def keyGen(self):
        return self.pke.keyGen()
    
    def encapsulate(self, pk):
        m = self.pke.Rq([choice([0, 1]) for i in range(self.pke.n)]) 
        key = self.H(dumps(m)[:12])
        y, c = self.pke.encryptCCA(dumps(m)[:12], pk)
        return y, c, key
    
    def decapsulate(self, y, c, pk, sk):
        m = self.pke.decryptCCA(y, c, pk, sk)
        return self.H(m)

## **Cenário de teste do KEM**

In [6]:
kem = KYBER_KEM(512)
pk, sk = kem.keyGen()

print("Encapsulating public key...")
y, c, key = kem.encapsulate(pk)

print("Decapsulating public key...")
decapsulated_key = kem.decapsulate(y, c, pk, sk)

if(key == decapsulated_key):
    print("Encapsulate and Decapsulate work!")
else:
    print("Something went wrong...")

Encapsulating public key...
Decapsulating public key...
Encapsulate and Decapsulate work!
